In [1]:
import numpy as np
import pandas as pd

GRID = 'grid'
MG = 'mg'
SHS = 'shs'
ELECTRIFICATION_OPTIONS = [GRID, MG, SHS]
BAU_SCENARIO = 'bau'
SE4ALL_SCENARIO = 'se4all'
SE4ALL_FLEX_SCENARIO = 'se4all_shift'
PROG_SCENARIO = 'prog'
SCENARIOS = [BAU_SCENARIO, SE4ALL_SCENARIO, PROG_SCENARIO]

# Names for display
SCENARIOS_DICT = {
    BAU_SCENARIO: 'BaU',
    SE4ALL_SCENARIO: 'SE4All',
    PROG_SCENARIO: 'prOG',
}

ELECTRIFICATION_DICT = {
    GRID: 'Grid',
    MG: 'Mini Grid',
    SHS: 'Solar Home System'
}

# column names of the exogenous results
POP_GET = ['pop_get_%s_2030' % opt for opt in ELECTRIFICATION_OPTIONS]
HH_GET = ['hh_get_%s_2030' % opt for opt in ELECTRIFICATION_OPTIONS]
HH_CAP = ['hh_%s_capacity' % opt for opt in ELECTRIFICATION_OPTIONS]
HH_SCN2 = ['hh_cap_scn2_%s_capacity' % opt for opt in ELECTRIFICATION_OPTIONS]
INVEST = ['%s_investment_cost' % opt for opt in [MG, SHS]]
INVEST_CAP = ['tier_capped_%s_investment_cost' % opt for opt in [MG, SHS]]
GHG = ['ghg_%s_2030' % opt for opt in ELECTRIFICATION_OPTIONS]
GHG_CAP = ['tier_capped_ghg_%s_2030' % opt for opt in ELECTRIFICATION_OPTIONS]
EXO_RESULTS = POP_GET + HH_GET + HH_CAP + HH_SCN2 + INVEST + INVEST_CAP + GHG + GHG_CAP

# source http://www.worldbank.org/content/dam/Worldbank/Topics/Energy%20and%20Extract/
# Beyond_Connections_Energy_Access_Redefined_Exec_ESMAP_2015.pdf
MIN_TIER_LEVEL = 3
MIN_RATED_CAPACITY = {1: 3, 2: 50, 3: 200, 4: 800, 5: 2000}  # index is TIER level [W]
MIN_ANNUAL_CONSUMPTION = {1: 4.5, 2: 73, 3: 365, 4: 1250, 5: 3000}  # index is TIER level [kWh/a]
RATIO_CAP_CONSUMPTION = {}

# Investment Cost Source: Arranz and Worldbank,
# BENCHMARKING STUDY OF SOLAR PV MINIGRIDS INVESTMENT COSTS, 2017 (Jabref)
# unit is USD per household
MEDIAN_INVESTMENT_COST = {1: 742, 2: 1273, 3: 2516, 4: 5277, 5: 5492}

# drives for the socio-economic model
IMPACT_FACTORS = pd.DataFrame(
    {
        MG: [3, 13. / 6, 19. / 6, 3.25, 11. / 3],
        SHS: [23. / 12, 4.5, 37. / 12, 17. / 6, 41. / 12],
        'labels': [
            'high_gdp',
            'high_mobile_money',
            'high_ease_doing_business',
            'low_corruption',
            'high_grid_weakness'
        ]
    }
)
IMPACT_FACTORS = IMPACT_FACTORS.set_index('labels')

MENTI_DRIVES = ['gdp', 'mobile_money', 'ease_doing_business', 'corruption', 'weak_grid']

# $RT_shift_factors.$P$2
WEIGHT_MENTIS = 0.2
# -->WEIGHT_GRID = 0.8 ($RT_shift_factors.$O$2)  and  WEIGHT_GRID = 1 - WEIGHT_MENTIS
RISE_INDICES = ['rise_%s' % opt for opt in ELECTRIFICATION_OPTIONS]
SHIFT_MENTI = ['shift_menti_mg', 'shift_menti_shs']

BASIC_ROWS = [
    'People share',
    'People (k)',
    'HH (k)',
    'HH cap. (MW)',
    'HH cap. (MW) (TIER + 1)',
    'Investment MUSD',
    'Investment (TIER + 1) MUSD',
]
# labels of the columns of the result tables
LABEL_COLUMNS = ELECTRIFICATION_DICT.copy()
# a column for the row labels
LABEL_COLUMNS['labels'] = ''
LABEL_COLUMNS['total'] = 'Total'
BASIC_COLUMNS_ID = ['labels'] + ELECTRIFICATION_OPTIONS + ['total']
GHG_COLUMNS_ID = ['labels'] + ELECTRIFICATION_OPTIONS + ['total']
COMPARE_COLUMNS_ID = ['labels']
for opt in ELECTRIFICATION_OPTIONS + ['total']:
    COMPARE_COLUMNS_ID.append(opt)
    COMPARE_COLUMNS_ID.append('comp_{}'.format(opt))


## Test TIER level attribution

In [ ]:
from data_preparation import _find_tier_level

for hh_cons in np.array([1, 50, 100, 400, 2000, 5000]):
    print(_find_tier_level(hh_cons, 2))

In [ ]:
from data_preparation import _slope_capacity_vs_yearly_consumption
RATIO_CAP_CONSUMPTION = {}
TIER_LEVELS = [1, 2, 3, 4, 5]
for tier_lvl in [1, 2, 3, 4]:
    RATIO_CAP_CONSUMPTION[tier_lvl] = _slope_capacity_vs_yearly_consumption(tier_lvl)
RATIO_CAP_CONSUMPTION

In [ ]:
MIN_RATED_CAPACITY = {1: 3, 2: 50, 3: 200, 4: 800, 5: 2000}  # index is TIER level [W]
MIN_ANNUAL_CONSUMPTION = {1: 4.5, 2: 73, 3: 365, 4: 1250, 5: 3000}  # index is TIER level [kWh/a]

In [ ]:
for tier_lvl in TIER_LEVELS:
    print(MIN_RATED_CAPACITY[tier_lvl] / MIN_ANNUAL_CONSUMPTION[tier_lvl])

In [ ]:
cap = [MIN_RATED_CAPACITY[i] for i in TIER_LEVELS]
df = pd.DataFrame(cap)
df[0].plot()

In [ ]:
cons = [MIN_ANNUAL_CONSUMPTION[i] for i in TIER_LEVELS]
df = pd.DataFrame(cons)
df[0].plot()

In [ ]:
from data_preparation import IMPACT_FACTORS
IMPACT_FACTORS.index.to_list()

In [ ]:
for opt in [MG, SHS]:
    for input_name in IMPACT_FACTORS.index.to_list():
        #print("Input('impact-{}-{}-input', 'value'),".format(opt, input_name.replace('_', '-')))
        print('impact_{}_{},'.format(opt, input_name.replace('high', '')))



In [ ]:
country,country_iso, ghg_grid_2030,ghg_mg_2030,ghg_shs_2030,ghg_no_access_2030,tier_capped_ghg_grid_2030,tier_capped_ghg_mg_2030,tier_capped_ghg_shs_2030,tier_capped_ghg_no_access_2030

# Tests of the model

In [2]:
from data.data_preparation import compute_ndc_results_from_raw_data
SCENARIOS_DATA = {
    sce: compute_ndc_results_from_raw_data(sce, MIN_TIER_LEVEL).to_json() for sce in SCENARIOS
}

/home/Pierre.Duc/Dokumente/repos/NDC_OG/data/data_preparation.py:319: RuntimeWarning: invalid value encountered in true_divide
  / shs_sales_volumes['tot_5-7'].values


In [ ]:
xls_bau = pd.read_csv('data/xls_bau.csv', float_precision='high')
xls_se = pd.read_csv('data/xls_se.csv', float_precision='high')
xls_prog = pd.read_csv('data/xls_prog.csv', float_precision='high')

invest_bau = pd.read_csv('data/invest_bau.csv', float_precision='high')
invest_se = pd.read_csv('data/invest_se.csv', float_precision='high')
invest_prog = pd.read_csv('data/invest_prog.csv', float_precision='high')

## Test Exogenous results

### BaU

In [ ]:
bau_df = pd.read_json(SCENARIOS_DATA[BAU_SCENARIO]).set_index('country_iso').sort_index(ascending=True)
xls_bau = pd.read_csv('data/xls_bau.csv', float_precision='high')

COMP_COLS = POP_GET + HH_GET + HH_CAP + HH_SCN2

df_diff = xls_bau[COMP_COLS] - bau_df[COMP_COLS]

def highlight_mismatch(col, eps=0.2):
    return df_diff.loc[np.abs(df_diff[col]) > eps]

l = []
for col in COMP_COLS:
    temp = highlight_mismatch(col).index.to_list()
    if temp:
        print('problems with ', col, temp)
        print(len(temp))
    l = l + temp
len(set(l))
l = list(set(l))
df_diff.loc[l]

### SE4ALL

In [ ]:
se_df = pd.read_json(SCENARIOS_DATA[SE4ALL_SCENARIO]).set_index('country_iso').sort_index(ascending=True)
xls_se = pd.read_csv('data/xls_se.csv', float_precision='high').set_index('country_iso').sort_index(ascending=True)

COMP_COLS = POP_GET + HH_GET + HH_CAP + HH_SCN2

df_diff = xls_se[COMP_COLS] - se_df[COMP_COLS]

def highlight_mismatch(col, eps=0.001):
    return df_diff.loc[np.abs(df_diff[col]) > eps]

l = []
for col in COMP_COLS:
    temp = highlight_mismatch(col).index.to_list()
    if temp:
        print('problems with ', col, temp)
        print(len(temp))
    l = l + temp
len(set(l))
l = list(set(l))
df_diff.loc[l]


### prOG

In [ ]:
xls_prog = pd.read_csv('data/xls_prog.csv', float_precision='high')
prog_df = pd.read_json(SCENARIOS_DATA[PROG_SCENARIO]).set_index('country_iso').sort_index(ascending=True)


COMP_COLS = POP_GET + HH_GET + HH_CAP + HH_SCN2

df_diff = xls_prog[COMP_COLS] - prog_df[COMP_COLS]

def highlight_mismatch(col, eps=0.2):
    return df_diff.loc[np.abs(df_diff[col]) > eps]

l = []
for col in COMP_COLS:
    temp = highlight_mismatch(col).index.to_list()
    if temp:
        print('problems with ', col, temp)
        print(len(temp))
    l = l + temp
len(set(l))
l = list(set(l))
df_diff.loc[l]

## Test GHG

### BaU

In [ ]:
bau_df = pd.read_json(SCENARIOS_DATA[BAU_SCENARIO]).set_index('country_iso').sort_index(ascending=True)
ghg_bau = pd.read_csv('data/ghg_bau.csv', float_precision='high').set_index('country_iso').sort_index(ascending=True)

COMP_COLS = GHG + GHG_CAP

df_diff = ghg_bau[COMP_COLS] - bau_df[COMP_COLS]

def highlight_mismatch(col, eps=40):
    return df_diff.loc[np.abs(df_diff[col]) > eps]

l = []
for col in COMP_COLS:
    temp = highlight_mismatch(col).index.to_list()
    if temp:
        print('problems with ', col, temp)
        print(len(temp))
    l = l + temp
len(set(l))
l = list(set(l))
df_diff.loc[l]

### SE4All

In [ ]:
se_df = pd.read_json(SCENARIOS_DATA[SE4ALL_SCENARIO]).set_index('country_iso').sort_index(ascending=True)
ghg_se = pd.read_csv('data/ghg_se.csv', float_precision='high').set_index('country_iso').sort_index(ascending=True)

COMP_COLS = GHG + GHG_CAP

df_diff = ghg_se[COMP_COLS] - se_df[COMP_COLS]

def highlight_mismatch(col, eps=40):
    return df_diff.loc[np.abs(df_diff[col]) > eps]

l = []
for col in COMP_COLS:
    temp = highlight_mismatch(col).index.to_list()
    if temp:
        print('problems with ', col, temp)
        print(len(temp))
    l = l + temp
len(set(l))
l = list(set(l))
df_diff.loc[l]

## Saved from BaU

In [ ]:
bau_df = pd.read_json(SCENARIOS_DATA[BAU_SCENARIO]).set_index('country_iso').sort_index(ascending=True)
se_df = pd.read_json(SCENARIOS_DATA[SE4ALL_SCENARIO]).set_index('country_iso').sort_index(ascending=True)

COMP_COLS = GHG + GHG_CAP

saved_se_df = bau_df[COMP_COLS] - se_df[COMP_COLS]


### prOG

In [ ]:
prog_df = pd.read_json(SCENARIOS_DATA[PROG_SCENARIO]).set_index('country_iso').sort_index(ascending=True)
ghg_prog = pd.read_csv('data/ghg_prog.csv', float_precision='high').set_index('country_iso').sort_index(ascending=True)

COMP_COLS = GHG + GHG_CAP

df_diff = ghg_prog[COMP_COLS] - prog_df[COMP_COLS]

def highlight_mismatch(col, eps=40):
    return df_diff.loc[np.abs(df_diff[col]) > eps]

l = []
for col in COMP_COLS:
    temp = highlight_mismatch(col).index.to_list()
    if temp:
        print('problems with ', col, temp)
        print(len(temp))
    l = l + temp
len(set(l))
l = list(set(l))
df_diff.loc[l]

## Test Investment cost

### BaU

In [ ]:
bau_df = pd.read_json(SCENARIOS_DATA[BAU_SCENARIO]).set_index('country_iso').sort_index(ascending=True)
invest_bau = pd.read_csv('data/invest_bau.csv', float_precision='high').set_index('country_iso').sort_index(ascending=True)

COMP_COLS = INVEST + INVEST_CAP

df_diff = invest_bau[COMP_COLS] - bau_df[COMP_COLS]

def highlight_mismatch(col, eps=40):
    return df_diff.loc[np.abs(df_diff[col]) > eps]

l = []
for col in COMP_COLS:
    temp = highlight_mismatch(col).index.to_list()
    if temp:
        print('problems with ', col, temp)
        print(len(temp))
    l = l + temp
len(set(l))
l = list(set(l))
df_diff.loc[l]

### SE4All

In [ ]:
se_df = pd.read_json(SCENARIOS_DATA[SE4ALL_SCENARIO]).set_index('country_iso').sort_index(ascending=True)
invest_se = pd.read_csv('data/invest_se.csv', float_precision='high').set_index('country_iso').sort_index(ascending=True)

COMP_COLS = INVEST + INVEST_CAP

df_diff = invest_se[COMP_COLS] - se_df[COMP_COLS]

def highlight_mismatch(col, eps=20):
    return df_diff.loc[np.abs(df_diff[col]) > eps]

l = []
for col in COMP_COLS:
    temp = highlight_mismatch(col).index.to_list()
    if temp:
        print('problems with ', col, temp)
        print(len(temp))
    l = l + temp
len(set(l))
l = list(set(l))
df_diff.loc[l]

### prOG

In [ ]:
prog_df = pd.read_json(SCENARIOS_DATA[PROG_SCENARIO]).set_index('country_iso').sort_index(ascending=True)
invest_prog = pd.read_csv('data/invest_prog.csv', float_precision='high').set_index('country_iso').sort_index(ascending=True)

COMP_COLS = INVEST + INVEST_CAP

df_diff = invest_prog[COMP_COLS] - prog_df[COMP_COLS]

def highlight_mismatch(col, eps=40):
    return df_diff.loc[np.abs(df_diff[col]) > eps]

l = []
for col in COMP_COLS:
    temp = highlight_mismatch(col).index.to_list()
    if temp:
        print('problems with ', col, temp)
        print(len(temp))
    l = l + temp
len(set(l))
l = list(set(l))
df_diff.loc[l]

# Rise shifts for electrification option

$i \in $ (grid, mg, shs)

$N_i$ : population getting option $i$ in 2030

$R_i$ : RISE score for the option $i$

$\delta_{ij} = R_i - R_j$

$\Delta_i = \frac{\sum_j \delta_ij}{\sum R_k} $

$\Delta N_i = \Delta_i (\sum_j N_j)$

Constraint is that $\sum_j \Delta N_j$

## Bulk case
$R_i \neq 0$, $\forall i$

$N_i - \Delta N_i > 0$, $\forall i$

In [45]:
df = pd.DataFrame(data=[[90,95,55], [30, 20, 55], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]], columns=ELECTRIFICATION_OPTIONS)
df.iloc[2] = df.iloc[1]/ df.iloc[1].sum()
df.iloc[3, 0] = (df.iloc[0].grid - df.iloc[0].mg) + (df.iloc[0].grid - df.iloc[0].shs)
df.iloc[3, 1] = (df.iloc[0].mg - df.iloc[0].grid) + (df.iloc[0].mg - df.iloc[0].shs)
df.iloc[3, 2] = (df.iloc[0].shs  - df.iloc[0].mg ) + (df.iloc[0].shs  - df.iloc[0].grid)

df.iloc[3] = df.iloc[3] / df.iloc[0].sum()

for j in range(3):
    df.iloc[4, j] = df.iloc[1].sum() * df.iloc[3,j]
    df.iloc[5, j] = df.iloc[4, j] / df.iloc[1,j]
    df.iloc[6, j] = df.iloc[1, j] + df.iloc[4, j]

df['sum'] = df.sum(axis=1)
df['labels'] = ['R_i', 'N_i' ,'n_i', 'Delta_i', 'Delta N_i', 'Delta N_i / N_i', 'Delta N_i + N_i']
df

,grid,mg,shs,sum,labels
0,90.000000,95.000000,55.000000,240.000000,R_i
1,30.000000,20.000000,55.000000,105.000000,N_i
2,0.285714,0.190476,0.523810,1.000000,n_i
3,0.125000,0.187500,-0.312500,0.000000,Delta_i
4,13.125000,19.687500,-32.812500,0.000000,Delta N_i
5,0.437500,0.984375,-0.596591,0.825284,Delta N_i / N_i
6,43.125000,39.687500,22.187500,105.000000,Delta N_i + N_i


## Limit case
$R_i \neq 0$, $\forall i$

$N_i - \Delta N_i < 0$,  for some $i$

if $N_i - \Delta N_i < 0$,  for one $i$ ([90,10,45], [30, 40, 30]), then we take $\Delta N_i = N_i \Delta_i < 0$ for this $i$ and we split this value between the $j$ for which $N_j - \Delta N_j > 0$, with the weight $\frac{R_j}{\sum_{k \neq i} R_k}$. That way we don't over-penalize

if $N_i - \Delta N_i < 0$,  for two $i$ ([90,10,15], [30, 40, 5]), then we take $\Delta N_i = N_i \Delta_i < 0$ for these $i$ and we add their  absolute sum to the $j$ for which $N_j - \Delta N_j > 0$. That way we don't over-penalize


The case ([90,10,45], [30, 700, 30]) penalize the R3 = 45 more than the R2=20 which is wrong, this is a limiting case to explore, maybe the solution would be to still penalize according to the initial $\Delta_{ij}$ (ie if $\Delta_{ij}$ < 0), then one must nevertheless be penalized, although it might be small and not have the penalty of the $i$ for which $N_i - \Delta N_i < 0$ redistributed equally among the remaining $j$

Maybe we want to cap the maximum value that can be given to a $N_i$ to be no larger than $N_i \Delta_i$, maybe not ?
between [90,5,55], [30, 600, 30] and [90,1,55], [30, 600, 30], the reward is dispoportionate in favor of R1 in first case

In [157]:
df = pd.DataFrame(data=[[90,5,55], [30, 600, 30], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]], columns=ELECTRIFICATION_OPTIONS)
df.iloc[2] = df.iloc[1]/ df.iloc[1].sum()
df.iloc[3, 0] = (df.iloc[0].grid - df.iloc[0].mg) + (df.iloc[0].grid - df.iloc[0].shs)
df.iloc[3, 1] = (df.iloc[0].mg - df.iloc[0].grid) + (df.iloc[0].mg - df.iloc[0].shs)
df.iloc[3, 2] = (df.iloc[0].shs  - df.iloc[0].mg ) + (df.iloc[0].shs  - df.iloc[0].grid)

df.iloc[3] = df.iloc[3] / df.iloc[0].sum()

for j in range(3):
    df.iloc[4, j] = df.iloc[1].sum() * df.iloc[3,j]
    df.iloc[5, j] = df.iloc[4, j] / df.iloc[1,j]
    df.iloc[6, j] = df.iloc[1, j] + df.iloc[4, j]

correct = True
    
if correct:
    diff = df.iloc[6].values
    diff = diff[diff < 0]
    if len(diff) == 2:
        print('There are two')
        diff = df.iloc[6].values
        diff = diff[diff > 0]
        idx = df.iloc[6].to_list().index(diff[0])

        dN = 0
        for i in range(3):
            if i != idx:
                df.iloc[6, i] = df.iloc[1, i] * df.iloc[3, i]
                dN = dN + np.abs(df.iloc[6, i])
        df.iloc[6, idx] = dN   
        #df.iloc[7] = df.iloc[6] + df.iloc[1]

    elif len(diff) == 1:
        print('one difference is smaller than 0')
        idx = df.iloc[6].to_list().index(diff[0])
        # eps = N_i + \Delta N_i < 0
        eps = df.iloc[1,idx] * df.iloc[3, idx]
        norm = 0
        idx2 = None
        for i in range(3):
            if i != idx:
                if df.iloc[3,i] < 0:
                    print('one of the remaing should have a penalty')
                    idx2 = i
                norm = norm + df.iloc[0,i]
        if idx2 is None:
            print('the difference will be fully split between the two other case')
            eps = df.iloc[1,idx]
            for i in range(3):
                if i == idx:
                    df.iloc[6,i] = -eps
                else:
                    df.iloc[6,i] = np.abs(eps) * df.iloc[0,i]/norm
        else:
            dN = 0
            for i in range(3):
                if i == idx or i == idx2:
                    df.iloc[6, i] = df.iloc[1, i] * df.iloc[3, i]
                    dN = dN + np.abs(df.iloc[6, i])
            
            for i in range(3):
                if i != idx and i !=idx2:
                    df.iloc[6, i] = dN   
         
        #df.iloc[7] = df.iloc[6] + df.iloc[1]

    elif len(df) == 3:
        print(diff)
        print('error, all values are negative')
    else:
        print('no difference is smaller than 0')
        correct = False
        
    df.iloc[7] = df.iloc[6] + df.iloc[1]
    for i in range(3):
        df.iloc[5, i] = df.iloc[6, i] / df.iloc[1,i]
        #df.iloc[6, i] = df.iloc[1, i] + df.iloc[4, i]
df['sum'] = df.sum(axis=1)
if correct:
    df['labels'] = ['R_i', 'N_i' ,'n_i', 'Delta_i', 'Delta N_i', 'Delta N_i / N_i (case 2)', 'Delta N_i (case 2)','Delta N_i + N_i (case 2)']
else:
    df['labels'] = ['R_i', 'N_i' ,'n_i', 'Delta_i', 'Delta N_i', 'Delta N_i / N_i', 'Delta N_i + N_i','']
df

no difference is smaller than 0


,grid,mg,shs,sum,labels
0,90.000000,5.000000,55.000000,1.500000e+02,R_i
1,30.000000,600.000000,30.000000,6.600000e+02,N_i
2,0.045455,0.909091,0.045455,1.000000e+00,n_i
3,0.800000,-0.900000,0.100000,2.775558e-17,Delta_i
4,528.000000,-594.000000,66.000000,0.000000e+00,Delta N_i
5,18.600000,0.010000,3.200000,2.181000e+01,Delta N_i / N_i
6,558.000000,6.000000,96.000000,6.600000e+02,Delta N_i + N_i
7,588.000000,606.000000,126.000000,1.320000e+03,


In [102]:
df = pd.DataFrame(data=[[16.67,69.17,22.22], [4.268328e+06, 3.457142e+05, 9.163149e+06], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]], columns=ELECTRIFICATION_OPTIONS)
df.iloc[2] = df.iloc[1]/ df.iloc[1].sum()
#df.iloc[3, 0] = (df.iloc[0].grid* df.iloc[2].grid - df.iloc[0].mg * df.iloc[2].mg) + (df.iloc[0].grid* df.iloc[2].grid - df.iloc[0].shs * df.iloc[2].shs)
#df.iloc[3, 1] = (df.iloc[0].mg * df.iloc[2].mg - df.iloc[0].grid* df.iloc[2].grid) + (df.iloc[0].mg * df.iloc[2].mg - df.iloc[0].shs * df.iloc[2].shs)
#df.iloc[3, 2] = (df.iloc[0].shs * df.iloc[2].shs - df.iloc[0].mg * df.iloc[2].mg) + (df.iloc[0].shs * df.iloc[2].shs - df.iloc[0].grid* df.iloc[2].grid)

df.iloc[3, 0] = (df.iloc[0].grid - df.iloc[0].mg) + (df.iloc[0].grid - df.iloc[0].shs)
df.iloc[3, 1] = (df.iloc[0].mg - df.iloc[0].grid) + (df.iloc[0].mg - df.iloc[0].shs)
df.iloc[3, 2] = (df.iloc[0].shs  - df.iloc[0].mg ) + (df.iloc[0].shs  - df.iloc[0].grid)

df.iloc[3] = df.iloc[3] / df.iloc[0].sum()

for j in range(3):
    df.iloc[4, j] = df.iloc[1].sum()*df.iloc[3,j]
print(df.iloc[2].sum())
print(df.iloc[3].sum())
print(df.iloc[4].sum())

df.iloc[5] = df.iloc[1] + df.iloc[4]

N = df.iloc[1,2] + df.iloc[4,2]
df.iloc[6,0] = N * df.iloc[0,0] / (df.iloc[0,0]+ df.iloc[0,1])
df.iloc[6,1] = N * df.iloc[0,1] / (df.iloc[0,0]+ df.iloc[0,1])
df.iloc[6,2] = -N

df.iloc[7] =  df.iloc[6] + df.iloc[4]

for j in range(3):
    df.iloc[6,j] = df.iloc[7,j] / df.iloc[1,j] 

df['sum'] = df.sum(axis=1)
df['labels'] = ['R_i', 'N_i' ,'n_i', 'Delta_i', 'Delta N_i', '+N_i', '+N_i', '']


df

1.0
-5.551115123125783e-17
-1.862645149230957e-09


,grid,mg,shs,sum,labels
0,1.667000e+01,6.917000e+01,2.222000e+01,1.080600e+02,R_i
1,4.268328e+06,3.457142e+05,9.163149e+06,1.377719e+07,N_i
2,3.098112e-01,2.509323e-02,6.650956e-01,1.000000e+00,n_i
3,-5.372016e-01,9.203220e-01,-3.831205e-01,-5.551115e-17,Delta_i
4,-7.401129e+06,1.267945e+07,-5.278324e+06,-1.862645e-09,Delta N_i
5,-3.132801e+06,1.302517e+07,3.884825e+06,1.377719e+07,+N_i
6,-1.557214e+00,4.573098e+01,-1.000000e+00,4.317377e+01,+N_i
7,-6.646701e+06,1.580985e+07,-9.163149e+06,0.000000e+00,


In [103]:
df = pd.DataFrame(data=[[90,90,65], [30, 20, 8], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]], columns=ELECTRIFICATION_OPTIONS)
df.iloc[2] = df.iloc[1]/ df.iloc[1].sum()

df.iloc[3, 0] = (df.iloc[0].grid - df.iloc[0].mg) + (df.iloc[0].grid - df.iloc[0].shs)
df.iloc[3, 1] = (df.iloc[0].mg - df.iloc[0].grid) + (df.iloc[0].mg - df.iloc[0].shs)
df.iloc[3, 2] = (df.iloc[0].shs  - df.iloc[0].mg ) + (df.iloc[0].shs  - df.iloc[0].grid)

df.iloc[3] = df.iloc[3] / df.iloc[0].sum()

for j in range(3):
    df.iloc[4, j] = df.iloc[1].sum()*df.iloc[3,j]

df.iloc[5] = df.iloc[1] + df.iloc[4]

N = df.iloc[1,2] + df.iloc[4,2]
df.iloc[6,0] = N * df.iloc[0,0] / (df.iloc[0,0]+ df.iloc[0,1])
df.iloc[6,1] = N * df.iloc[0,1] / (df.iloc[0,0]+ df.iloc[0,1])
df.iloc[6,2] = -N

df.iloc[7] =  df.iloc[6] + df.iloc[4]

#for j in range(3):
#    df.iloc[6,j] = df.iloc[7,j] / df.iloc[1,j] 

df['sum'] = df.sum(axis=1)
df['labels'] = ['R_i', 'N_i' ,'n_i', 'Delta_i', 'Delta N_i', '+N_i', '+N_i', '']


df

,grid,mg,shs,sum,labels
0,90.000000,90.000000,65.000000,245.0,R_i
1,30.000000,20.000000,8.000000,58.0,N_i
2,0.517241,0.344828,0.137931,1.0,n_i
3,0.102041,0.102041,-0.204082,0.0,Delta_i
4,5.918367,5.918367,-11.836735,0.0,Delta N_i
5,35.918367,25.918367,-3.836735,58.0,+N_i
6,-1.918367,-1.918367,3.836735,0.0,+N_i
7,4.000000,4.000000,-8.000000,0.0,


In [ ]:
df = pd.DataFrame(data=[[90,70,70], [30, 20, 2], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]], columns=ELECTRIFICATION_OPTIONS)
df.iloc[2] = df.iloc[1]/ df.iloc[1].sum()
#df.iloc[3, 0] = (df.iloc[0].grid* df.iloc[2].grid - df.iloc[0].mg * df.iloc[2].mg) + (df.iloc[0].grid* df.iloc[2].grid - df.iloc[0].shs * df.iloc[2].shs)
#df.iloc[3, 1] = (df.iloc[0].mg * df.iloc[2].mg - df.iloc[0].grid* df.iloc[2].grid) + (df.iloc[0].mg * df.iloc[2].mg - df.iloc[0].shs * df.iloc[2].shs)
#df.iloc[3, 2] = (df.iloc[0].shs * df.iloc[2].shs - df.iloc[0].mg * df.iloc[2].mg) + (df.iloc[0].shs * df.iloc[2].shs - df.iloc[0].grid* df.iloc[2].grid)

df.iloc[3, 0] = (df.iloc[0].grid - df.iloc[0].mg) + (df.iloc[0].grid - df.iloc[0].shs)
df.iloc[3, 1] = (df.iloc[0].mg - df.iloc[0].grid) + (df.iloc[0].mg - df.iloc[0].shs)
df.iloc[3, 2] = (df.iloc[0].shs  - df.iloc[0].mg ) + (df.iloc[0].shs  - df.iloc[0].grid)

df.iloc[3] = df.iloc[3] / df.iloc[0].sum()

for j in range(3):
    df.iloc[4, j] = df.iloc[1].sum()*df.iloc[3,j]
print(df.iloc[2].sum())
print(df.iloc[3].sum())
print(df.iloc[4].sum())

df.iloc[5] = df.iloc[1] + df.iloc[4]

N = df.iloc[1,2]
df.iloc[6,0] = df.iloc[1,2] - df.iloc[4,1]
df.iloc[6,1] = df.iloc[4,1]
df.iloc[6,2] = -N

df.iloc[7] =  df.iloc[6] + df.iloc[1]

#for j in range(3):
#    df.iloc[6,j] = df.iloc[7,j] / df.iloc[1,j] 

df['sum'] = df.sum(axis=1)
df['labels'] = ['R_i', 'N_i' ,'n_i', 'Delta_i', 'Delta N_i', '+N_i', '+N_i', '']


df

In [ ]:
df = pd.DataFrame(data=[[90,75,70], [30, 20, 2], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]], columns=ELECTRIFICATION_OPTIONS)
df.iloc[2] = df.iloc[1]/ df.iloc[1].sum()
#df.iloc[3, 0] = (df.iloc[0].grid* df.iloc[2].grid - df.iloc[0].mg * df.iloc[2].mg) + (df.iloc[0].grid* df.iloc[2].grid - df.iloc[0].shs * df.iloc[2].shs)
#df.iloc[3, 1] = (df.iloc[0].mg * df.iloc[2].mg - df.iloc[0].grid* df.iloc[2].grid) + (df.iloc[0].mg * df.iloc[2].mg - df.iloc[0].shs * df.iloc[2].shs)
#df.iloc[3, 2] = (df.iloc[0].shs * df.iloc[2].shs - df.iloc[0].mg * df.iloc[2].mg) + (df.iloc[0].shs * df.iloc[2].shs - df.iloc[0].grid* df.iloc[2].grid)

df.iloc[3, 0] = (df.iloc[0].grid - df.iloc[0].mg) + (df.iloc[0].grid - df.iloc[0].shs)
df.iloc[3, 1] = (df.iloc[0].mg - df.iloc[0].grid) + (df.iloc[0].mg - df.iloc[0].shs)
df.iloc[3, 2] = (df.iloc[0].shs  - df.iloc[0].mg ) + (df.iloc[0].shs  - df.iloc[0].grid)

df.iloc[3] = df.iloc[3] / df.iloc[0].sum()

for j in range(3):
    df.iloc[4, j] = df.iloc[1].sum()*df.iloc[3,j]
print(df.iloc[2].sum())
print(df.iloc[3].sum())
print(df.iloc[4].sum())

df.iloc[5] = df.iloc[1] + df.iloc[4]

N = np.max([df.iloc[1,2] + df.iloc[4,2], -df.iloc[1,2]])
print(N)
print()
df.iloc[6,0] = N * df.iloc[0,0] / (df.iloc[0,0]+ df.iloc[0,1])
df.iloc[6,1] = N * df.iloc[0,1] / (df.iloc[0,0]+ df.iloc[0,1])
df.iloc[6,2] = -N
df.iloc[7] =  df.iloc[6] + df.iloc[4]

#for j in range(3):
#    df.iloc[6,j] = df.iloc[7,j] / df.iloc[1,j] 

df['sum'] = df.sum(axis=1)
df['labels'] = ['R_i', 'N_i' ,'n_i', 'Delta_i', 'Delta N_i', '+N_i', '+N_i', '']


df

Case where $\Delta N_i =  N_i \Delta_i$, then the rest $\varepsilon = \sum \Delta N_i = \sum_i N_i \Delta_i \neq 0$ is redistributed to the prorata of the $n_i$ --> $\Delta N'_i =  \Delta N_i - \varepsilon n_i$ such that $\sum \Delta' N_i = 0$

In [ ]:
df = pd.DataFrame(data=[[90,15,15], [30, 20, 10], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]], columns=ELECTRIFICATION_OPTIONS)
df.iloc[2] = df.iloc[1]/ df.iloc[1].sum()

df.iloc[3, 0] = (df.iloc[0].grid - df.iloc[0].mg) + (df.iloc[0].grid - df.iloc[0].shs)
df.iloc[3, 1] = (df.iloc[0].mg - df.iloc[0].grid) + (df.iloc[0].mg - df.iloc[0].shs)
df.iloc[3, 2] = (df.iloc[0].shs  - df.iloc[0].mg ) + (df.iloc[0].shs  - df.iloc[0].grid)

df.iloc[3] = df.iloc[3] / df.iloc[0].sum()

for j in range(3):
    df.iloc[4, j] =  df.iloc[3,j] * df.iloc[1,j]#* df.iloc[1].sum()

eps = df.iloc[4].sum()
for j in range(3):
    df.iloc[5, j] = df.iloc[4,j] - dN * df.iloc[2,j]
    df.iloc[6, j] = df.iloc[5, j] / df.iloc[1,j]
    
df['sum'] = df.sum(axis=1)
df['labels'] = ['R_i', 'N_i' ,'n_i', 'Delta_i', 'Delta N_i', 'Delta N_i (case 2)', 'Delta N_i / N_i (case 2)']
df